<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Part-I:-Analytical-Option-Formulae" data-toc-modified-id="Part-I:-Analytical-Option-Formulae-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Part I: Analytical Option Formulae</a></span><ul class="toc-item"><li><span><a href="#Bachelier" data-toc-modified-id="Bachelier-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Bachelier</a></span></li><li><span><a href="#Black-Scholes" data-toc-modified-id="Black-Scholes-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Black-Scholes</a></span></li><li><span><a href="#Black76" data-toc-modified-id="Black76-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Black76</a></span></li><li><span><a href="#Displaced-diffusion" data-toc-modified-id="Displaced-diffusion-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Displaced-diffusion</a></span></li></ul></li><li><span><a href="#Part-II:-Model-Calibration" data-toc-modified-id="Part-II:-Model-Calibration-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Part II: Model Calibration</a></span></li><li><span><a href="#Part-III:-Static-Replication" data-toc-modified-id="Part-III:-Static-Replication-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Part III: Static Replication</a></span></li><li><span><a href="#Part-IV-(Dynamic-Hedging)" data-toc-modified-id="Part-IV-(Dynamic-Hedging)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Part IV (Dynamic Hedging)</a></span></li></ul></div>

In [4]:
import numpy as np
from scipy.stats import norm

# Part I: Analytical Option Formulae

Consider the following European options:
- Vanilla call/put
- Digital cash-or-nothing call/put
- Digital asset-or-nothing call/put

Derive and implement the following models to value these options in Python:
1. Black-Scholes model
2. Bachelier model
3. Black76 model
4. Displaced-diffusion model

## Bachelier

1. Vanilla Call Option
$$
    V_0^c = (S_0-K)\Phi\left(\frac{S_0-K}{S_0\sigma\sqrt{T}}\right) + S_0\sigma\sqrt{T}\phi\left(\frac{S_0-K}{S_0\sigma\sqrt{T}}\right)
$$
2. Vanilla Put Option
$$
    V_0^p = (K-S_0)\Phi\left(\frac{K-S_0}{S_0\sigma\sqrt{T}}\right) + S_0\sigma\sqrt{T}\phi\left(\frac{K-S_0}{S_0\sigma\sqrt{T}}\right)
    \\\\
    V_0^p = (K-S_0) + V_0^c
$$
3. Digital cash-or-nothing call
$$
    V_0^{conc} = \Phi\left(\frac{S_0-K}{S_0\sigma\sqrt{T}}\right)
$$
4. Digital cash-or-nothing put
$$
    V_0^{conp} = \Phi\left(\frac{K-S_0}{S_0\sigma\sqrt{T}}\right) = 1 - V_0^{dc}
$$
5. Digital asset-or-nothing call
$$
    V_0^{aonc} = S_0\Phi\left(\frac{S_0-K}{S_0\sigma\sqrt{T}}\right) + S_0\sigma\sqrt{T}\phi\left(\frac{S_0-K}{S_0\sigma\sqrt{T}}\right)
$$
6. Digital asset-or-nothing put
$$
    V_0^{aonp} = S_0\Phi\left(\frac{K-S_0}{S_0\sigma\sqrt{T}}\right) - S_0\sigma\sqrt{T}\phi\left(\frac{K-S_0}{S_0\sigma\sqrt{T}}\right) = S_0 - V_0^{aonc}
$$

In [5]:
def bachelier(So, K, sigma, T, option_type):
    '''
    Calculate European option price using Bachelier model:
        dSt = sigma * S0 * dWt 
        St = S0*(1 + sigma*Wt)
    
    Parameter
    ---------
    So: float
        price of underlying asset at time 0
    K: float
        strike price of option
    sigma: float
        variance of Brownian motion
    T: float
        length of time
    option_type: str
        type of European option. 
        Including: van call/put (vanilla), con call/put (cash-or-nothing), aon call/put (asset-or-nothing)
    
    Return
    ------
    val: value of the option at time 0
    '''
    
    
    xs = (K-So) / (So * sigma * np.sqrt(T))
    val = None
    
    if So == K:
        return sigma*So*np.sqrt(T/(2*np.pi))
    
    if option_type == 'van call':
        val = (So - K) * norm.cdf(-xs) + So*sigma*np.sqrt(T)*norm.pdf(-xs)
    elif option_type == 'van put':
        val = (K - So) * norm.cdf(xs) + So*sigma*np.sqrt(T)*norm.pdf(xs)
    elif option_type == 'con call':
        val = norm.cdf(-xs)
    elif option_type == 'con put':
        val = norm.cdf(xs)
    elif option_type == 'aon call':
        val = So*norm.cdf(-xs) + So*sigma*np.sqrt(T)*norm.pdf(-xs)
    elif option_type == 'aon put':
        val = So*norm.cdf(xs) - So*sigma*np.sqrt(T)*norm.pdf(xs)
    else:
        raise(ValueError("Option type is invalid. " +
                         "Should be either 'van call', 'van put', 'con call', 'con put', 'aon call', or 'aon put'"))
    
    return val
                

In [6]:
So = 50
K = 20
sigma=0.2
T = 4

In [7]:
print("Vanilla call:", bachelier(So, K, sigma, T, option_type='van call'))
print("Vanilla put:", bachelier(So, K, sigma, T, option_type='van put'))
print("Cash-or-nothing call:", bachelier(So, K, sigma, T, option_type='con call'))
print("Cash-or-nothing put:", bachelier(So, K, sigma, T, option_type='con put'))
print("Asset-or-nothing call:", bachelier(So, K, sigma, T, option_type='aon call'))
print("Asset-or-nothing put:", bachelier(So, K, sigma, T, option_type='aon put'))

Vanilla call: 30.586135875252094
Vanilla put: 0.5861358752520927
Cash-or-nothing call: 0.9331927987311419
Cash-or-nothing put: 0.06680720126885807
Asset-or-nothing call: 49.24999184987493
Asset-or-nothing put: 0.7500081501250686


## Black-Scholes

$$
d_1 = \frac{log{\frac{S_0}{K}} + (r+\frac{\sigma^2}{2})T}{\sigma\sqrt{T}}
$$

$$
d_2 = \frac{log{\frac{S_0}{K}} + (r-\frac{\sigma^2}{2})T}{\sigma\sqrt{T}}
$$

1. Vanilla Call Option
$$
    V_0^c = S_0\Phi(d_1) - Ke^{-rT}\Phi(d_2)
$$
2. Vanilla Put Option
$$
    V_0^p = -S_0\Phi(-d_1) + Ke^{-rT}\Phi(-d_2) = Ke^{-rT} - S_0 + V_0^c
$$
3. Digital cash-or-nothing call
$$
    V_0^{conc} = e^{-rT}\Phi(d_2)
$$
4. Digital cash-or-nothing put
$$
    V_0^{conp} = e^{-rT}\Phi(-d_2) = e^{-rT} - V_0^{conc}
$$
5. Digital asset-or-nothing call
$$
    V_0^{aonc} = S_0\Phi(d_1)
$$
6. Digital asset-or-nothing put
$$
    V_0^{aonp} = S_0\Phi(-d_1) = S_0 - V_0^{aonc}
$$

In [11]:
def black_scholes(So, K, r, sigma, T, option_type):
    '''
    Calculate European option price using Black-Scholes (1973) model:
        dSt = r*dSt + sigma*St*dWt 
        St = S0*exp{(r-sigma^2/2)t + sigma*Wt}
    
    Parameter
    ---------
    So: float
        price of underlying asset at time 0
    K: float
        strike price of option
    r: float
        drift of St
    sigma: float
        variance of Brownian motion
    T: float
        length of time
    option_type: str
        type of European option. 
        Including: van call/put (vanilla), con call/put (cash-or-nothing), aon call/put (asset-or-nothing)
    
    Return
    ------
    val: value of the option at time 0
    '''
    
    
    d1 = (np.log(So/K) + (r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = (np.log(So/K) + (r-sigma**2/2)*T) / (sigma*np.sqrt(T))
    val = None
    
    if So == K:
        return sigma*So*np.sqrt(T/(2*np.pi))
    
    if option_type == 'van call':
        val = So*norm.cdf(d1) - K*np.e**(-r*T)*norm.cdf(d2)
    elif option_type == 'van put':
        val = -So*norm.cdf(-d1) + K*np.e**(-r*T)*norm.cdf(-d2)
    elif option_type == 'con call':
        val = np.e**(-r*T) * norm.cdf(d2)
    elif option_type == 'con put':
        val = np.e**(-r*T) * norm.cdf(-d2)
    elif option_type == 'aon call':
        val = So*norm.cdf(d1)
    elif option_type == 'aon put':
        val = So*norm.cdf(-d1)
    else:
        raise(ValueError("Option type is invalid. " +
                         "Should be either 'van call', 'van put', 'con call', 'con put', 'aon call', or 'aon put'"))
    
    return val
                

In [12]:
So = 50
K = 20
r = 0.13
sigma=0.2
T = 4

In [13]:
print("Vanilla call:", black_scholes(So, K, r, sigma, T, option_type='van call'))
print("Vanilla put:", black_scholes(So, K, r, sigma, T, option_type='van put'))
print("Cash-or-nothing call:", black_scholes(So, K, r, sigma, T, option_type='con call'))
print("Cash-or-nothing put:", black_scholes(So, K, r, sigma, T, option_type='con put'))
print("Asset-or-nothing call:", black_scholes(So, K, r, sigma, T, option_type='aon call'))
print("Asset-or-nothing put:", black_scholes(So, K, r, sigma, T, option_type='aon put'))

Vanilla call: 38.1099781201697
Vanilla put: 0.0003890795735903797
Cash-or-nothing call: 0.5943133351354707
Cash-or-nothing put: 0.00020721283472371576
Asset-or-nothing call: 49.99624482287911
Asset-or-nothing put: 0.003755177120883936


## Black76

## Displaced-diffusion

# Part II: Model Calibration

Calibrate the following models to match the option prices:
1. Displaced-diffusion model
2. SABR model (fix β = 0.8)

Plot the fitted implied volatility smile against the market data.

Report the model parameters: 
1. σ,β
2. α,ρ,ν

Discuss how does change β in the displaced-diffusion model and ρ, ν in the SABR model affect the shape of the implied volatility smile.

# Part III: Static Replication

Determine the price of these 2 derivative contracts if we use:
1. Black-Scholes model (what σ should we use?)
2. Bachelier model (what σ should we use?)
3. Static-replication of European payoff (using the SABR model calibrated in the previous question)

# Part IV (Dynamic Hedging)

Suppose $S_0$ = 100, σ=0.2, r=5\%, T = 1 month, K = 100. 

Use a Black-Scholes model to simulate the stock price. Suppose we sell this at-the-money call option, and we hedge N times during the life of the call option. Assume there are 21 trading days over the month.
